In [1]:
# make sure the transformers package is installed on your machine
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 12.2 MB/s 
     |████████████████████████████████| 6.6 MB 62.9 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 596 kB 54.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Dataset analysis

In [2]:
# NEEDED
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()  # default parameters and configuration for BERT


In [3]:
# NEEDED
# Load the fast tokenizer from saved file
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
import os
from google.colab import drive
import sys 

drive.mount('/content/gdrive')
sys.path.append('drive/gdrive/MyDrive/training_set.json')


Mounted at /content/gdrive


In [5]:
%cd gdrive/MyDrive

/content/gdrive/MyDrive


In [6]:
# NEEDED
with open('training_set.json','rb') as f:
     squad = json.load(f)

In [ ]:
squad.keys()

dict_keys(['data', 'version'])

In [ ]:
type(squad['data'])

list

In [ ]:
print(len(squad['data']))

442


In [ ]:
type(squad['data'][0])

dict

In [ ]:
squad['data'][0].keys()

dict_keys(['title', 'paragraphs'])

In [ ]:
type(squad['data'][1]['title'][0])

str

In [ ]:
type(squad['data'][0]['paragraphs'][0])

dict

In [ ]:
squad['data'][0]['paragraphs'][0].keys()

dict_keys(['context', 'qas'])

In [ ]:
type(squad['data'][0]['paragraphs'][0]['context'])

str

In [ ]:
type(squad['data'][0]['paragraphs'][0]['context'][0])

str

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0])

dict

In [ ]:
squad['data'][0]['paragraphs'][0]['qas'][0].keys()

dict_keys(['answers', 'question', 'id'])

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0] ['answers'][0])

dict

In [ ]:
squad['data'][0]['paragraphs'][0]['qas'][0] ['answers'][0].keys()

dict_keys(['answer_start', 'text'])

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0] ['id'][0])

str

In [ ]:
type(squad['data'][0]['paragraphs'][0]['qas'][0] ['question'][0])

str

POINT 1 : 'squad' is a complicated dictionary with two keys 'data' and 'version'. 

POINT 2 : With respect to our work at hand, the value of the key'version' need not play any role at all. 

POINT 3:  All expected analysis, will spin around the value corresponding to the 'data' key.

POINT 4: The value corresponding to 'data' key is itself a list comprising 442 elements. 
Since we need to split the given data set into training and validation, it is at the 'data' key level we possibly have to do the splitting.

POINT 5: The 'data', which is of type list, the elements of that list is again a dictionary having two keys 'title' and 'paragraphs'.

POINT 6: 'title' is of type string whereas'paragraphs' is again of type dictionary with two keys 'context' and 'qas'.

POINT 7:  'context' is of type strings and 'qas' is again a dictionary.

POINT 8: The dictionary 'qas' contains three keys namely, 'answers','question' and 'id'.'id' and 'questions' are strings whereas 'answers' is again another dictionary with keys 'answer_start' and 'text'.

POINT 9 : Also note, in the original data set we do not have any answer_end index as would be required as we use BERT or variants for building the Q/A application


In [30]:
def count_questions_multiple_answers(data):
  count = 0
  for group in data:
      for passage in group['paragraphs']:
          context = passage['context']
          for qa in passage['qas']:
              question = qa['question']
              all_answers = [_["text"] for _ in qa["answers"]]
              if len(all_answers) > 1:
                count += 1
  return count

In [32]:
count = count_questions_multiple_answers(squad["data"])
print(count)

0


From the output of the last cell we can easily conclude that each question has only one answer. So technically we would need to create some sort of pseudo data to tackle the questions-with-multiple-answers scenario.

# Pre-processing
In this section we fine-tune a pre-trained version of BERT after correctly preparing the data so that it can be fed to the model.

In [7]:
# NEEDED
import math
import random

# split dataset in train and validation sets according to the title of the paragraphs

random.Random(42).shuffle(squad["data"])
train_data_list = squad['data'][:math.floor(0.8*len(squad['data']))]
val_data_list = squad['data'][math.floor(0.8*len(squad['data'])):]

print(len(train_data_list + val_data_list) == len(squad['data']))

True


In [8]:
# NEEDED
train_data = {"data": train_data_list}
val_data = {"data": val_data_list}

In [9]:
# NEEDED
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        # context = My name is Liam
        # answer = Liam
        # question = What is my name?
        # before the loop: is_char_in_ans = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        # after the loop: is_char_in_ans = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        # tokenized_context = [123, 456, 789, 111]
        # tokenized_context.offsets = [(0, 2), (3, 7), (8, 10), (11, 14)]
        # ans_token_idx = [4]
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        # start_token_idx = 4
        # end_token_idx = 4 
        # there is only one token in the answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        #tokenized_question = [97, 46, 32, 10]
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        # input_ids = [123, 456, 789, 111, 97, 46, 32, 10]
        # token_type_ids = [0, 0, 0, 0, 1, 1, 1, 1]
        # attention_mask = [1, 1, 1, 1, 1, 1, 1, 1]
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        # max_len = 10
        # padding_length = 10 - 8 = 2
        # input_ids = [123, 456, 789, 111, 97, 46, 32, 10, 0, 0]
        # token_type_ids = [0, 0, 0, 0, 1, 1, 1, 1, 0, 0]
        # attention_mask = [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets

def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_input_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


train_squad_examples = create_squad_examples(train_data)
x_train, y_train = create_input_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

val_squad_examples = create_squad_examples(val_data)
x_val, y_val = create_input_targets(val_squad_examples)
print(f"{len(val_squad_examples)} validation points created.")

70223 training points created.
17376 validation points created.


In [75]:
from typing import Set
# NEEDED
def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text

def prepare_sample(tokenizer, question, context):
    tokenized_context = tokenizer.encode(context)
    tokenized_question = tokenizer.encode(question)

    # tokenize question and text as a pair
    input_ids = tokenized_context.ids + tokenized_question.ids[1:]
  
    token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
    
    attention_mask = [1] * len(input_ids)

    padding_length = max_len - len(input_ids)
    if padding_length > 0:  # pad
      input_ids = input_ids + ([0] * padding_length)
      token_type_ids = token_type_ids + ([0] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)

    return input_ids, token_type_ids, attention_mask

def question_answer(k, df, index, model, tokenizer):
    input_ids = df.loc[index].at["input_ids"]
    token_type_ids = df.loc[index].at["token_type_ids"]
    attention_mask = df.loc[index].at["attention_mask"]

    # from input_ids extract the tokens belonging to the context
    context_tokens = [ii for tt, ii in zip(token_type_ids, input_ids) if tt == 0]
    # decode the context to encode it again to obtain a variable of type "Encoding"
    context = tokenizer.decode(context_tokens)
    tokenized_context = tokenizer.encode(context)

    data = [np.array([input_ids]), np.array([token_type_ids]), np.array([attention_mask])]
    
    start, end = model.predict(data)
    offsets = tokenized_context.offsets
    all_answers = []
    top_idxs_start = np.argpartition(start[0], -k)[-k:]
    top_idxs_end = np.argpartition(end[0], -k)[-k:]
    for start, end in zip(top_idxs_start, top_idxs_end):
      # access the "offsets" attribute of the class "Encoding"
      pred_char_start = offsets[start][0]
      if end < len(offsets):
          pred_char_end = offsets[end][1]
          pred_ans = context[pred_char_start:pred_char_end]
      else:
          pred_ans = context[pred_char_start:]
      all_answers.append(pred_ans)
    return all_answers

# Fine tuning
We fine tune the pre-trained bert model. These cells have to be run only once.

In [ ]:
def create_model():
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("bert-base-uncased", return_dict=True)

    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model


In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.83.231.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.83.231.138:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_3[0][0]',          

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
layer_names = [layer.name for layer in model.layers]
print(layer_names)

['input_1', 'input_3', 'input_2', 'tf_bert_model_1', 'start_logit', 'end_logit', 'flatten', 'flatten_1', 'activation', 'activation_1']


In [ ]:
class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_val, y_val):
        self.x_val = x_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_val)
        count = 0
        eval_examples_no_skip = [_ for _ in val_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_val[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")


When using a GPU the batch size should be 8 in order to avoid "ResourceExhausted" errors. Using a GPU one epoch requires at least 45 minutes (the end was not reached due to impatience).

In [ ]:
exact_match_callback = ExactMatch(x_val, y_val)
model.fit(
    x_train,
    y_train,
    epochs=6,  # 3 epochs are recommended
    verbose=2,
    batch_size=64,
    callbacks=[exact_match_callback]
)


Epoch 1/6



epoch=1, exact match score=0.66
1078/1078 - 330s - loss: 2.6500 - activation_loss: 1.3849 - activation_1_loss: 1.2651 - 330s/epoch - 306ms/step
Epoch 2/6

epoch=2, exact match score=0.67
1078/1078 - 228s - loss: 1.6029 - activation_loss: 0.8509 - activation_1_loss: 0.7521 - 228s/epoch - 212ms/step
Epoch 3/6

epoch=3, exact match score=0.67
1078/1078 - 228s - loss: 1.1264 - activation_loss: 0.6048 - activation_1_loss: 0.5215 - 228s/epoch - 211ms/step
Epoch 4/6

epoch=4, exact match score=0.67
1078/1078 - 228s - loss: 0.8012 - activation_loss: 0.4314 - activation_1_loss: 0.3697 - 228s/epoch - 211ms/step
Epoch 5/6

epoch=5, exact match score=0.66
1078/1078 - 228s - loss: 0.5815 - activation_loss: 0.3134 - activation_1_loss: 0.2681 - 228s/epoch - 212ms/step
Epoch 6/6

epoch=6, exact match score=0.65
1078/1078 - 228s - loss: 0.4344 - activation_loss: 0.2324 - activation_1_loss: 0.2020 - 228s/epoch - 211ms/step


In [ ]:
question_answer(model, tokenizer, question, context)

NameError: ignored

# Download model

Here we download the model to avoid having to train it every time using a TPU (which is not always available). These cells have to be run only once.

In [ ]:
%cd ../../

/content


In [ ]:
!mkdir -p saved_model

In [ ]:
%cd saved_model/

/content/saved_model


In [ ]:
model.save('my_model.h5')

In [ ]:
!du -sh my_model.h5

1.3G	my_model.h5


In [ ]:
from google.colab import files

files.download('my_model.h5')

# Load fine tuned model
We load the fine tuned module and use the function "question_answer" to build the .json file with the predictions.

In [11]:
# NEEDED
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5', custom_objects={'TFBertModel': TFBertModel})

In [ ]:
new_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 384)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_1[0][0]',                
                                thPoolingAndCrossAt               'input_3[0][0]',          

In [12]:
import pandas as pd

def prepare_samples_df(tokenizer, raw_data):
  # build the pandas dataframe incrementally by adding dictionaries (rows) to a list (the dataframe)
  dataset = []
  for item in raw_data["data"]:
    for para in item["paragraphs"]:
        context = para["context"]
        for qa in para["qas"]:
            dataset_elem = {}
            question = qa["question"]
            dataset_elem["id"] = qa["id"]
            input_ids, token_type_ids, attention_mask = prepare_sample(tokenizer, 
                                                                        question, 
                                                                        context)
            dataset_elem["input_ids"] = input_ids
            dataset_elem["token_type_ids"] = token_type_ids
            dataset_elem["attention_mask"] = attention_mask
            
            dataset.append(dataset_elem)
  return pd.DataFrame(dataset)

In [13]:
df = prepare_samples_df(tokenizer, val_data)
df.head()

id  \
0  572a8d2834ae481900deab79   
1  572a8d2834ae481900deab7a   
2  572a8d2834ae481900deab7b   
3  572a8d2834ae481900deab7c   
4  572a8e0834ae481900deab81   

                                           input_ids  \
0  [101, 1999, 29478, 1010, 2520, 9502, 2631, 199...   
1  [101, 1999, 29478, 1010, 2520, 9502, 2631, 199...   
2  [101, 1999, 29478, 1010, 2520, 9502, 2631, 199...   
3  [101, 1999, 29478, 1010, 2520, 9502, 2631, 199...   
4  [101, 2241, 2006, 1996, 2714, 12363, 14128, 19...   

                                      token_type_ids  \
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                      attention_mask  
0  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
2  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
3  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...  
4  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...

In [ ]:
print(df.shape)

(17376, 4)


In [14]:
for i in range(50):
    input_ids = df.loc[i].at["input_ids"]
    token_type_ids = df.loc[i].at["token_type_ids"]
    
    question_tokens = [ii for tt, ii in zip(token_type_ids, input_ids) if tt == 1]
    print("ID: {} QUESTION: {}".format(df.loc[i].at["id"], tokenizer.decode(question_tokens)))

ID: 572a8d2834ae481900deab79 QUESTION: who founded philadelphia?
ID: 572a8d2834ae481900deab7a QUESTION: what role did philadelphia play in the american revolution?
ID: 572a8d2834ae481900deab7b QUESTION: what purpose did philadelphia serve while d. c. was under construction?
ID: 572a8d2834ae481900deab7c QUESTION: when did it reach 2 million occupants >
ID: 572a8e0834ae481900deab81 QUESTION: what was the historical economic base of philadelphia?
ID: 572a8e0834ae481900deab82 QUESTION: when did the city near bankruptcy?
ID: 572a8e0834ae481900deab83 QUESTION: where did wealthy residents move?
ID: 572a8e0834ae481900deab84 QUESTION: who replaced the wealthy?
ID: 572a8ebc34ae481900deab89 QUESTION: what is the gdp of the city?
ID: 572a8ebc34ae481900deab8a QUESTION: where does it rank in comparison to other us cities in gdp?
ID: 572a8ebc34ae481900deab8b QUESTION: how many fortune 500 companies call philadelpia home?
ID: 572a8ebc34ae481900deab8c QUESTION: how many domestic tourists visit annually

In [76]:
# NEEDED
def compute_predictions(df, model, tokenizer):
  # iterate over the dataframe and predict an answer for each sample
  pred_dict = {}
  for i in range(50):
    pred_dict[df.loc[i].at["id"]] = question_answer(2, df, i, model , tokenizer)
  pred_json = json.dumps(pred_dict, indent = 4)
  return pred_json

In [77]:
pred_json = compute_predictions(df, new_model, tokenizer)

In [78]:
print(pred_json)

{
    "572a8d2834ae481900deab79": [
        "",
        "william penn"
    ],
    "572a8d2834ae481900deab7a": [
        "",
        "instrumental role"
    ],
    "572a8d2834ae481900deab7b": [
        "",
        "temporary u. s. capital"
    ],
    "572a8d2834ae481900deab7c": [
        "",
        "1950"
    ],
    "572a8e0834ae481900deab81": [
        "historically been manufacturing,",
        "manufacturing"
    ],
    "572a8e0834ae481900deab82": [
        "late 1980s.",
        "1980s"
    ],
    "572a8e0834ae481900deab83": [
        "suburbs and more immigrants moved into the city",
        "nearby suburbs"
    ],
    "572a8e0834ae481900deab84": [
        "wealthier residents moved into the nearby suburbs and more immigrants moved into the city",
        "immigrants"
    ],
    "572a8ebc34ae481900deab89": [
        "388 billion,",
        "$ 388 billion"
    ],
    "572a8ebc34ae481900deab8a": [
        "fourth in the nation",
        "ninth"
    ],
    "572a8ebc34ae481900deab8b":